In [1]:
from tensorflow.keras.models import Sequential, model_from_json
json_file = open('Model/CNN_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("Model/CNN_model_weights.h5")
print("Loaded model from disk")

Loaded model from disk


In [2]:
import pickle

with open('Model/scaler2.pickle', 'rb') as f:
    scaler2 = pickle.load(f)
    
with open('Model/encoder2.pickle', 'rb') as f:
    encoder2 = pickle.load(f)

    
print("Done")   

Done


In [3]:
import librosa
def zcr(data,frame_length,hop_length):
    zcr=librosa.feature.zero_crossing_rate(data,frame_length=frame_length,hop_length=hop_length)
    return np.squeeze(zcr)
def rmse(data,frame_length=2048,hop_length=512):
    rmse=librosa.feature.rms(data,frame_length=frame_length,hop_length=hop_length)
    return np.squeeze(rmse)
def mfcc(data,sr,frame_length=2048,hop_length=512,flatten:bool=True):
    mfcc=librosa.feature.mfcc(data,sr=sr)
    return np.squeeze(mfcc.T)if not flatten else np.ravel(mfcc.T)

def extract_features(data,sr=22050,frame_length=2048,hop_length=512):
    result=np.array([])
    
    result=np.hstack((result,
                      zcr(data,frame_length,hop_length),
                      rmse(data,frame_length,hop_length),
                      mfcc(data,sr,frame_length,hop_length)
                     ))
    return result

In [4]:
import numpy as np
import librosa
def get_predict_feat(path):
    d, s_rate= librosa.load(path, duration=2.5, offset=0.6)
    res=extract_features(d)
    features=np.array(res)
    result= np.zeros(2376)
    result[:features.size] = features
    result=np.reshape(result,newshape=(1,2376))
    i_result = scaler2.transform(result)
    final_result=np.expand_dims(i_result, axis=2)
    
    return final_result

In [5]:
emotions1={1:'Neutral', 2:'Calm', 3:'Happy', 4:'Sad', 5:'Angry', 6:'Fear', 7:'Disgust',8:'Surprise'}
def prediction(path1):
    res=get_predict_feat(path1)
    predictions=loaded_model.predict(res)
    y_pred = encoder2.inverse_transform(predictions)  
    result = y_pred[0][0]
    return result

In [6]:
def get_first_time(input_string):
    # Find the first underscore
    first_underscore_index = input_string.find('_')
    
    # Find the second underscore after the first one
    second_underscore_index = input_string.find('_', first_underscore_index + 1)
    
    # Find the first period after the second underscore
    if second_underscore_index != -1:
        # Find the first hyphen
        first_hyphen_index = input_string.find('-')
    
        # Find the second hyphen after the first one
        second_hyphen_index = input_string.find('-', first_hyphen_index + 1)
        
        # Extract the substring between the second underscore and the first period
        if second_hyphen_index != -1:
            return input_string[second_underscore_index + 1:second_hyphen_index]
    
    # Return None if the conditions are not met
    return None

In [7]:
def get_second_time(input_string):
    # Find the first hyphen
    first_hyphen_index = input_string.find('-')
    
    # Find the second hyphen after the first one
    second_hyphen_index = input_string.find('-', first_hyphen_index + 1)
    
    # Find the first period
    first_period_index = input_string.find('.')
    
    # Find the second period after the first one
    second_period_index = input_string.find('.', first_period_index + 1)
    
    # Find the second period after the first one
    third_period_index = input_string.find('.', second_period_index + 1)
    
    # Extract the substring between the second hyphen and the second period
    if second_hyphen_index != -1 and third_period_index != -1:
        return input_string[second_hyphen_index + 1:third_period_index]
    
    # Return None if the conditions are not met
    return None

In [8]:
import os
import pandas as pd
directory = "Segmentation/IWM42_20231213/"
directory_list = os.listdir(directory)

emotion = []
file_name = []
good = 0
bad = 0

for dir in directory_list: 
    directories = os.listdir(directory + dir)
    for file in directories:
        if ".wav" in file:
            time1 = float(get_second_time(file))
            time2 = float(get_first_time(file))
            if (time1 - time2) >= 1.000:
                file_name.append(file)
                emotion.append(prediction(directory+ dir + '/' + file))

result = pd.DataFrame(columns=['File_name', 'Emotions'])
result['Emotions'] = emotion
result['File_name'] = file_name
result.to_csv(directory+'emotionReults.csv', index=False)   
print(result.Emotions.value_counts())

disgust     91
happy       49
sad         44
neutral     32
angry       31
fear        16
surprise     5
Name: Emotions, dtype: int64


In [9]:
import os
import pandas as pd
directory = "Segmentation/IWT_20231212/"
directory_list = os.listdir(directory)

emotion = []
file_name = []
good = 0
bad = 0

for dir in directory_list: 
    directories = os.listdir(directory + dir)
    for file in directories:
        if ".wav" in file:
            time1 = float(get_second_time(file))
            time2 = float(get_first_time(file))
            if (time1 - time2) >= 1.000:
                file_name.append(file)
                emotion.append(prediction(directory+ dir + '/' + file))

result = pd.DataFrame(columns=['File_name', 'Emotions'])
result['Emotions'] = emotion
result['File_name'] = file_name
result.to_csv(directory+'emotionReults.csv', index=False) 
print(result.Emotions.value_counts())

neutral     148
angry       127
sad          96
happy        89
disgust      44
fear         30
surprise      2
Name: Emotions, dtype: int64


In [10]:
import os
import pandas as pd
directory = "Segmentation/IWT_20231213/"
directory_list = os.listdir(directory)

emotion = []
file_name = []
good = 0
bad = 0

for dir in directory_list: 
    directories = os.listdir(directory + dir)
    for file in directories:
        if ".wav" in file:
            time1 = float(get_second_time(file))
            time2 = float(get_first_time(file))
            if (time1 - time2) >= 1.000:
                file_name.append(file)
                emotion.append(prediction(directory+ dir + '/' + file))

result = pd.DataFrame(columns=['File_name', 'Emotions'])
result['Emotions'] = emotion
result['File_name'] = file_name
result.to_csv(directory+'emotionReults.csv', index=False)  
print(result.Emotions.value_counts())

angry       185
happy       145
sad          72
neutral      70
disgust      50
fear         41
surprise     14
Name: Emotions, dtype: int64


In [11]:
import os
import pandas as pd
directory = "Segmentation/IWTM42_20231215/"
directory_list = os.listdir(directory)

emotion = []
file_name = []
good = 0
bad = 0

for dir in directory_list: 
    directories = os.listdir(directory + dir)
    for file in directories:
        if ".wav" in file:
            time1 = float(get_second_time(file))
            time2 = float(get_first_time(file))
            if (time1 - time2) >= 1.000:
                file_name.append(file)
                emotion.append(prediction(directory+ dir + '/' + file))

result = pd.DataFrame(columns=['File_name', 'Emotions'])
result['Emotions'] = emotion
result['File_name'] = file_name
result.to_csv(directory+'emotionReults.csv', index=False)
print(result.Emotions.value_counts())

sad         91
neutral     75
disgust     66
happy       38
fear        24
angry       18
surprise     1
Name: Emotions, dtype: int64
